In [41]:
import os 
from mxnet import gluon
from mxnet import *
from mxnet.gluon import nn
import zipfile 
import tarfile 
import pandas as pd

In [42]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [43]:
print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


In [44]:
print(train_data.iloc[0:4, [0,1,2,3,-3,-2,-1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [45]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))
all_features

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


### DATA PREPROCESSING

In [46]:
numeric_features =  all_features.dtypes[all_features.dtypes != 'object'].index

#NORMALIZASYON
all_features[numeric_features] = all_features[numeric_features].apply(
lambda x: (x - x.mean()) / (x.std()))

#FILL MISSING VALUE
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [47]:
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(2919, 331)

In [48]:
n_train = train_data.shape[0]
train_features = np.array(all_features[:n_train].values, dtype= np.float32)
test_features = np.array(all_features[n_train:].values, dtype= np.float32)

train_labels = np.array(train_data.SalePrice.values, dtype= np.float32).reshape(-1,1)

### TRAINING

In [49]:
loss = gluon.loss.L2Loss()

def get_net():
    net = nn.Sequential()
    net.add(nn.Dense(1))
    net.initialize()
    return net

In [50]:
def log_rmse(net, features, labels):
    clipped_preds = np.clip(net(features), 1, float('inf'))
    return np.sqrt(2 * loss(np.log(clipped_preds), np.log(labels)).mean())

In [51]:
def train(net, train_features, train_labels, test_features, test_labels, num_epochs, 
         learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': learning_rate, 'wd': weight_decay})
    
    for epoch in range(num_epochs):
        for X, y in train_iter:
            with autograd.record():
                l = loss(net(X), y)
            l.backward()
            trainer.step(batch_size)
        train_ls.append(log_rmse(net, train_features, train_labels))
        
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return test_ls, test_ls

In [52]:
import numpy
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    
    for j in range(k):
        idx = slice(j * fold_size, (j+1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = numpy.concatenate((X_train, X_part), axis= 0)
            y_train = numpy.concatenate((y_train, y_part), axis= 0)
    return X_train, y_train, X_valid, y_valid

In [53]:
def k_fold(k ,X_train, y_train, num_epochs, 
          learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0 
    
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate, weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_l[-1]
        
        if i == 0:
            d2l.plot(list(range(1, num_epochs+1)), [train_ls, valid_ls],
                    xlabel= 'epoch', ylabel= 'rmse',
                    legend=['train', 'valid'], yscale= 'log')
        
        print('fold %d, train_rmse: %f, valid_rmse: %f' %(i, train_ls[-1], valid_ls[-1]))
        
    return train_l_sum / k, valid_l_sum / k 

### MODEL SELECTION

In [ ]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr, 
                        weight_decay, batch_size)

print('%d-fold validation: avg train rmse: %f, avg valid rmse: %f' % (k, train_l, valid_l))

### PREDICT AND SUBMIT

In [ ]:
def trai_and_pred(train_features, test_features, train_labels, test_data,
                 num_epochs, lr, weight_decay, batch_size):
    net = get_net()
    train_ls, _ = train(net, train_features, train_labels, None, None,
                       num_epochs, lr, weight_deecay, batch_size)
    d2l.plot(np.arange(1, num_epochs+1), [train_ls], xlabel= 'epoch',
            ylabel= 'rmse', yscale= 'log')
    print('train rmse %f' % train_ls[-1])
    
    preds = net(test_features).asnumpy()
    
    test_data['SalePrice'] = pd.Series(preds.reshape(1, -1)[0])
    
    submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
    submission.to_csv('submission.csv', index=False)
    
    train_and_pred(train_features, test_features, train_labels, test_data,
                  num_epochs, lr, weight_decay, batch_size)